In [4]:
import ipywidgets as w

In [42]:
intro = w.VBox(children=(w.HTML(value='<font size="+2"><b>CARA</b> Covid Calculator</font>'),
                         w.Text(placeholder="E.g. Workshop without masks", description="Simulation name:",
                                style={'description_width': '35%'}),
                         w.Text(placeholder="E.g. 17/R-033", description="Room number:",
                                style={'description_width': '35%'})))

intro

In [34]:
volume_text = w.Text(placeholder="Room volume (m³)", description="Room volume")
area_text = w.Text(placeholder="Room floor area (m²)", description="Floor area")
height_text = w.Text(placeholder="Room ceiling height (m²)", description="Ceiling height")

room_dimensions_box = w.VBox(children=(volume_text, w.Label(value="------- OR -------"),area_text, height_text),
                             layout=w.Layout(align_items='center'))
def manage_text_fields(b):
    volume_text.disabled = bool(area_text.value) or bool(height_text.value)
    area_text.disabled = bool(volume_text.value)
    height_text.disabled = bool(volume_text.value)

for text in [volume_text, area_text, height_text]:
    text.observe(manage_text_fields)
    text.style.description_width = "50%"

volume_text.observe(manage_text_fields)
area_text.observe(manage_text_fields)
height_text.observe(manage_text_fields)

room_dimensions_box